In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os

In [0]:
os.chdir('./drive/My Drive/Colab Notebooks/Voice Recognition/')

FileNotFoundError: ignored

In [0]:
normal = os.listdir('./Files/')
len(normal)

129

In [0]:
import pandas as pd
import librosa as lr
import matplotlib.pyplot as plt
import glob
%matplotlib inline
import librosa.display

In [0]:
audios=normal[:5]
audio_data=[]
for audio in audios:
  data,sr=lr.load('./Files/'+str(audio))
  audio_data.append((str(audio),data,sr))

In [0]:
audio_data[:3]

[('Copy of PTT-20190716-WA0040.wav',
  array([-1.9844701e-06,  3.4617603e-06, -2.5397916e-05, ...,
         -6.3064875e-04, -1.4380443e-04,  0.0000000e+00], dtype=float32),
  22050),
 ('Copy of PTT-20190716-WA0041.wav',
  array([-1.9844701e-06,  3.4617603e-06, -2.5397916e-05, ...,
         -1.1813657e-03, -1.3976368e-03,  0.0000000e+00], dtype=float32),
  22050),
 ('Copy of PTT-20190716-WA0042.wav',
  array([-1.9844701e-06,  3.4617603e-06, -2.5397916e-05, ...,
         -7.3041413e-03, -6.9063604e-03,  0.0000000e+00], dtype=float32),
  22050)]

In [0]:
len(audio_data)

5

In [0]:
import IPython.display as ipd
ipd.Audio('./Files/'+audio_data[4][0])

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [0]:
for ad in audio_data:
  plt.figure(figsize=(12,4))
  print(ad[0])
  ipd.Audio('./Files/'+ad[0])
  librosa.display.waveplot(ad[1],sr=ad[2])
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Part 2: Building better models<br>
Now let us see how we can leverage the concepts we learned above to solve the problem. We will follow these steps to solve the problem.

1.Load audio files<br>
2.Extract features from audio<br>
3.Convert the data to pass it in our deep learning model<br>
4.Run a deep learning model and get results<br>

In [0]:
bagunnava=os.listdir('./Baagunnaava/')
namaskaram = os.listdir('./Namskaram')
swagatham = os.listdir('./swagatham')

In [0]:
len(bagunnava); print('\n');len(normal); print('\n');len(namaskaram); print('\n');len(swagatham)

140

129

154

159

In [0]:
#labels=[1]*len(normal)
#len(labels)
#print(labels)

In [0]:
labels = []
#labels.extend([1]*len(bagunnava))
for j in range(len(normal)):
  labels.append('1')
for i in range(len(bagunnava)):
  labels.extend('2')
for k in range(len(namaskaram)):
  labels.extend('3')
for l in range(len(swagatham)):
  labels.extend('4')

In [0]:
len(labels)

582

In [0]:
df = pd.DataFrame()

In [0]:
df['Files'] = normal+bagunnava+namaskaram+swagatham

In [0]:
df['labels'] = labels
df.head()

,Files,labels
0,Copy of PTT-20190716-WA0040.wav,1
1,Copy of PTT-20190716-WA0041.wav,1
2,Copy of PTT-20190716-WA0042.wav,1
3,Copy of PTT-20190716-WA0046.wav,1
4,Copy of PTT-20190716-WA0043.wav,1


In [0]:
df.to_csv('data.csv',index=False)

In [0]:
ls

'Audio (1).ipynb'   data.csv                    Files/       svc_classifier.pkl
 Audio.ipynb        data.gsheet                 Namskaram/   swagatham/
 Baagunnaava/      'Feature Extraction.ipynb'   Others/      Testing.ipynb


In [0]:
data = pd.read_csv('data.csv')
data.head()

,Files,labels
0,Copy of PTT-20190716-WA0040.wav,1
1,Copy of PTT-20190716-WA0041.wav,1
2,Copy of PTT-20190716-WA0042.wav,1
3,Copy of PTT-20190716-WA0046.wav,1
4,Copy of PTT-20190716-WA0043.wav,1


# Feature Extraction

## Zero Crossing Rate
- The zero crossing rate is the rate of sign-changes along a signal, i.e., the rate at which the signal changes from positive to negative or back. This feature has been used heavily in both speech recognition and music information retrieval. It usually has higher values for highly percussive sounds like those in metal and rock.

In [0]:
zcr=librosa.zero_crossings(audio_data[0][1],pad=False)

In [0]:
sum(zcr)

1161

In [0]:
def zero_crossings(audio_path):
    data,sampling_rate=librosa.load(audio_path)
    return librosa.zero_crossings(data,pad=False)

## Spectral Centroid
- It indicates where the ”centre of mass” for a sound is located and is calculated as the weighted mean of the frequencies present in the sound. If the frequencies in music are same throughout then spectral centroid would be around a centre and if there are high frequencies at the end of sound then the centroid would be towards its end.

In [0]:
spectral_centroids=librosa.feature.spectral_centroid(audio_data[0][1],audio_data[0][2])
spectral_centroids

array([[1560.78107884, 1288.0069091 , 1178.88851171, 1272.57450306,
        1334.86037988, 1344.94245421, 1323.12480903, 1255.52438584,
        1434.38301205, 2552.35075742, 2585.04024814, 1438.92201743,
         883.50965854,  708.09932388,  796.78761489,  831.00483901,
         815.04263187,  934.6523208 , 1059.70257514, 1023.92248607,
        1211.26406584, 1249.95656581,  827.01434534,  561.73641542,
         445.39793407,  403.73615201,  392.81964551,  403.72449056,
         445.00421884,  737.99161264, 1885.7667755 , 1595.6229142 ,
         932.2766122 ,  725.03292506,  625.13580567,  558.54606623,
         691.24561445, 1090.02535316,  834.18531042,  765.44697145,
         788.89773656,  787.9500373 ,  734.12190224,  631.49415028,
         561.39901711,  509.61338259,  474.49158058,  484.93572073,
         550.79547124,  748.25495695,  851.79858541,  926.35460073,
         862.08965756,  821.16592704,  801.36853264,  932.27633174]])

### .spectral_centroid is used to calculate the spectral centroid for each frame. So it’ll return an array with columns equal to a number of frames present in your sample.

In [0]:
def get_spectral_centroid(audio):
    x,sr=librosa.load(audio)
    return librosa.feature.spectral_centroid(x,sr=sr)

## Spectral Roll off
 - Spectral rolloff is the frequency below which a specified percentage of the total spectral energy, e.g. 85%, lies.

It also gives results for each frame.

In [0]:
audio_data[0][1];audio_data[0][2]

array([-1.9844701e-06,  3.4617603e-06, -2.5397916e-05, ...,
       -6.3064875e-04, -1.4380443e-04,  0.0000000e+00], dtype=float32)

22050

In [0]:
spectral_rolloff = librosa.feature.spectral_rolloff(audio_data[0][1],sr=audio_data[0][2])[0]
spectral_rolloff

array([3402.24609375, 2648.58398438, 2314.81933594, 2562.45117188,
       2734.71679688, 2422.48535156, 2519.38476562, 2444.01855469,
       3111.54785156, 5523.26660156, 5577.09960938, 3488.37890625,
       1227.39257812,  915.16113281, 1195.09277344, 1356.59179688,
       1291.9921875 , 1679.58984375, 1787.25585938, 1819.55566406,
       2433.25195312, 2874.68261719, 1808.7890625 ,  699.82910156,
        624.46289062,  613.69628906,  613.69628906,  592.16308594,
        592.16308594, 1119.7265625 , 3919.04296875, 3736.01074219,
       1507.32421875,  689.0625    ,  624.46289062,  602.9296875 ,
       1087.42675781, 1927.22167969, 1087.42675781, 1022.82714844,
       1227.39257812, 1399.65820312, 1195.09277344,  979.76074219,
        775.1953125 ,  689.0625    ,  581.39648438,  613.69628906,
        667.52929688, 1173.55957031, 1528.85742188, 1701.12304688,
       1485.79101562, 1356.59179688, 1270.45898438, 1658.05664062])

In [0]:
def get_spectral_rolloff(audio):
    x,sr=librosa.load(audio)
    return librosa.feature.spectral_rolloff(x,sr=sr)

## MFCC — Mel-Frequency Cepstral Coefficients
- This feature is one of the most important method to extract a feature of an audio signal and is used majorly whenever working on audio signals. The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral envelope.

In [0]:
def get_mfcc(audio):
  x,sr=librosa.load(audio)
  return librosa.feature.mfcc(x,sr=sr)

In [0]:
df.head()

,Files,labels
0,Copy of PTT-20190716-WA0040.wav,1
1,Copy of PTT-20190716-WA0041.wav,1
2,Copy of PTT-20190716-WA0042.wav,1
3,Copy of PTT-20190716-WA0046.wav,1
4,Copy of PTT-20190716-WA0043.wav,1


In [0]:
bagun_urls=[]
for i in bagunnava:
  bagun_urls.append('./Baagunnaava/'+i)

In [0]:
bagun_urls[:2]

['./Baagunnaava/2019-07-16-11-34-49-224_Nervous.wav',
 './Baagunnaava/AUDIO-2019-07-15-18-04-44.wav']

In [0]:
other_urls=[]
nam_urls=[]
swag_urls=[]
for i in normal:
  other_urls.append('./Files/'+i)
for j in namaskaram:
  nam_urls.append('./Namskaram/'+j)
for k in swagatham:
  swag_urls.append('./swagatham/'+k)

In [0]:
swag_urls[:2]

['./swagatham/10.wav', './swagatham/12.wav']

In [0]:
urls=bagun_urls+other_urls+nam_urls+swag_urls

In [0]:
df['URLs'] = urls

In [0]:
df.head()

,Files,labels,URLs
0,Copy of PTT-20190716-WA0040.wav,1,./Baagunnaava/2019-07-16-11-34-49-224_Nervous.wav
1,Copy of PTT-20190716-WA0041.wav,1,./Baagunnaava/AUDIO-2019-07-15-18-04-44.wav
2,Copy of PTT-20190716-WA0042.wav,1,./Baagunnaava/AUDIO-2019-07-15-18-06-45.wav
3,Copy of PTT-20190716-WA0046.wav,1,./Baagunnaava/AUDIO-2019-07-15-20-58-23 2.wav
4,Copy of PTT-20190716-WA0043.wav,1,./Baagunnaava/AUDIO-2019-07-15-20-54-56.wav


In [0]:
df=df[df.Files.str.endswith('.wav')]

In [0]:
df.shape

(556, 3)

In [0]:
mfccs=df.URLs.apply(get_mfcc)

In [0]:
zcs=df.URLs.apply(zero_crossings)

In [0]:
scs=df.URLs.apply(get_spectral_centroid)

In [0]:
srs=df.URLs.apply(get_spectral_rolloff)

In [0]:
df['mfcc']=mfccs
df['zero_crossings']=zcs
df['spectral_centroid']=scs
df['spectral_rolloff']=srs

In [0]:
df.head(3)

,Files,labels,URLs,mfcc,zero_crossings,spectral_centroid,spectral_rolloff
0,Copy of PTT-20190716-WA0040.wav,1,./Baagunnaava/2019-07-16-11-34-49-224_Nervous.wav,"[[-786.9571835979152, -605.9462486937209, -206...","[False, False, False, False, False, False, Fal...","[[4521.962059945186, 4473.667548459104, 4497.3...","[[8301.0498046875, 8247.216796875, 8010.351562..."
1,Copy of PTT-20190716-WA0041.wav,1,./Baagunnaava/AUDIO-2019-07-15-18-04-44.wav,"[[-591.8722917412459, -489.6062341715169, -431...","[False, True, True, True, True, True, True, Tr...","[[2111.3505132893724, 2035.6576433925786, 2363...","[[3542.2119140625, 3542.2119140625, 4188.20800..."
2,Copy of PTT-20190716-WA0042.wav,1,./Baagunnaava/AUDIO-2019-07-15-18-06-45.wav,"[[-705.8256207820194, -705.8256207820194, -705...","[False, True, True, True, True, True, False, T...","[[3077.2910739759113, 3507.545327250375, 3363....","[[6535.3271484375, 7138.2568359375, 7138.25683..."


In [0]:
def get_spec_bandwidth(audio):
    x,sr=librosa.load(audio)
    return librosa.feature.spectral_bandwidth(y=x,sr=sr)

In [0]:
def get_rmse(audio):
    x,sr=librosa.load(audio)
    return librosa.feature.rms(y=x)

In [0]:
def get_chroma_stft(audio):
    x,sr=librosa.load(audio)
    return librosa.feature.chroma_stft(y=x,sr=sr)

In [0]:

def get_audio_len(audio):
  x,sr = librosa.load(audio)
  number_of_samples= x.shape[0]
  length_of_audio=round(( number_of_samples / sr ),2)
  return length_of_audio

In [0]:
def get_amplitude(audio):
    x,sr=librosa.load(audio)
    X = librosa.stft(x)
    return librosa.amplitude_to_db(abs(X))

In [0]:
def get_melspectrogram(audio):
  x,sr=librosa.load(audio)
  return librosa.feature.melspectrogram(y=x, sr=sr, n_mels=128,fmax=8000)

In [0]:
def get_chromagram(audio):
  x,sr=librosa.load(audio)
  return librosa.feature.chroma_cens(y=x, sr=sr)

In [0]:
def get_audio_data(audio):
  x,sr = librosa.load(audio)
  return x
def get_freqency(audio):
  x,sr = librosa.load(audio)
  return sr

In [0]:
get_freqency('./Baagunnaava/2019-07-16-11-34-49-224_Nervous.wav')


22050

In [0]:
df['spectral_bandwidth']=df.URLs.apply(get_spec_bandwidth)

In [0]:
df['rmse']=df.URLs.apply(get_rmse)

In [0]:
df['chroma_stft']=df.URLs.apply(get_chroma_stft)

In [0]:
df['Audio_length'] = df.URLs.apply(get_audio_len)
df['Audio_Data'] = df.URLs.apply(get_audio_data)
df['Frequency'] = df.URLs.apply(get_freqency)
df['Chromagram'] = df.URLs.apply(get_chromagram)
df['MelSpectrohram'] = df.URLs.apply(get_melspectrogram)
df['Amplitude_db'] = df.URLs.apply(get_amplitude)

In [0]:
import numpy as np
data['Frequency'] = df.Frequency
data['Audio_length'] = df.Audio_length
df['Audio_Data'] = df.Audio_Data.apply(np.mean)
data['zcr_mean']=df.zero_crossings.apply(np.mean)
data['sp_cen_mean']=df.spectral_centroid.apply(np.mean)
data['sp_rf_mean']=df.spectral_rolloff.apply(np.mean)
data['mfcc_mean']=df.mfcc.apply(np.mean)
data['spec_bw_mean']=df.spectral_bandwidth.apply(np.mean)
data['rmse_mean']=df.rmse.apply(np.mean)
data['chroma_stft_mean']=df.chroma_stft.apply(np.mean)
data['Chromagram'] = df.Chromagram.apply(np.mean)
data['MelSpectrohram'] = df.MelSpectrohram.apply(np.mean)
data['Amplitude_db'] = df.Amplitude_db.apply(np.mean)

In [0]:
data.head(3)

,Files,labels,Frequency,Audio_length,zcr_mean,sp_cen_mean,sp_rf_mean,mfcc_mean,spec_bw_mean,rmse_mean,chroma_stft_mean,Chromagram,MelSpectrohram,Amplitude_db
0,Copy of PTT-20190716-WA0040.wav,1,22050.0,1.70,0.095385,1557.936608,2977.110827,-21.310974,1446.468570,0.007986,0.461959,0.276743,0.042042,-45.591785
1,Copy of PTT-20190716-WA0041.wav,1,22050.0,1.79,0.115664,1774.966989,3151.163424,-14.459980,1608.826526,0.036079,0.299946,0.260970,0.822725,-30.595629
2,Copy of PTT-20190716-WA0042.wav,1,22050.0,2.62,0.117430,1853.480595,3594.329888,-15.858596,1635.967540,0.018241,0.338664,0.271582,0.203850,-36.165131


In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 14 columns):
Files               582 non-null object
labels              582 non-null int64
Frequency           556 non-null float64
Audio_length        556 non-null float64
zcr_mean            556 non-null float64
sp_cen_mean         556 non-null float64
sp_rf_mean          556 non-null float64
mfcc_mean           556 non-null float64
spec_bw_mean        556 non-null float64
rmse_mean           556 non-null float64
chroma_stft_mean    556 non-null float64
Chromagram          556 non-null float64
MelSpectrohram      556 non-null float64
Amplitude_db        556 non-null float64
dtypes: float64(12), int64(1), object(1)
memory usage: 63.7+ KB


In [0]:
#data.to_csv('data.csv',index=False)

# model Building

In [0]:
X = data.iloc[:,2:]
X.keys()

Index(['Frequency', 'Audio_length', 'zcr_mean', 'sp_cen_mean', 'sp_rf_mean',
       'mfcc_mean', 'spec_bw_mean', 'rmse_mean', 'chroma_stft_mean',
       'Chromagram', 'MelSpectrohram', 'Amplitude_db'],
      dtype='object')

In [0]:
Y = data.labels


In [0]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

x=scaler.fit_transform(X)


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_test,y_train,y_test=train_test_split(x,Y,test_size=0.2)

# XGBoost 

In [0]:
from xgboost import XGBClassifier

In [0]:
xg = XGBClassifier()
xg_model = xg.fit(x_train,y_train)

In [0]:
y_pred = xg_model.predict(x_test)
y_pred

array([2, 3, 4, 3, 3, 4, 2, 3, 1, 2, 1, 4, 2, 2, 1, 2, 1, 2, 4, 2, 3, 4,
       3, 3, 1, 2, 1, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 1, 2, 1, 2, 4, 4, 4,
       3, 4, 4, 2, 4, 3, 3, 2, 4, 3, 1, 1, 3, 2, 2, 1, 2, 4, 4, 1, 3, 3,
       4, 2, 2, 3, 3, 4, 2, 4, 2, 4, 2, 2, 3, 3, 2, 1, 3, 2, 1, 4, 2, 2,
       3, 2, 3, 3, 3, 3, 3, 3, 4, 1, 3, 2, 4, 1, 4, 2, 3, 3, 1, 2, 4, 4,
       2, 4, 4, 3, 4, 3, 4])

In [0]:
from sklearn import metrics

In [0]:
xg_cm = metrics.confusion_matrix(y_test,y_pred)
xg_cm

array([[14,  2,  3, 12],
       [ 0, 27,  2,  0],
       [ 2,  4, 20,  4],
       [ 1,  5,  6, 15]])

In [0]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.82      0.45      0.58        31
           2       0.71      0.93      0.81        29
           3       0.65      0.67      0.66        30
           4       0.48      0.56      0.52        27

    accuracy                           0.65       117
   macro avg       0.67      0.65      0.64       117
weighted avg       0.67      0.65      0.64       117



In [0]:
from sklearn.externals import joblib

In [0]:
joblib.dump(svm_model,'svc_classifier.pkl')

In [0]:
import pickle
pkl_filename = "svc_classifier.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(svc_classifier, file)
